In [393]:
%cd

/Users/jakbuck


/Users/jakbuck/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/Users/jakbuck/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:428: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [394]:
!python3 -m pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [395]:
import os

In [396]:

import pandas as pd
import re

In [397]:
df = pd.read_csv(os.path.expanduser("~/Documents/GitHub/q1-cs/foodstuffs/emerald.csv"))

In [398]:
df

,Item Name,What kind of item is it?,Total number of ingredients,Name of first (main) ingredient,Serving Size,Calories per serving,Total Sodium,Total Carbohydrates,Total Fat,Total Protein,What allergens does it contain?,Does it contain GMO crops?,Where was it made?,How Processed is it?\n(see instructions for ranking)
0,Milk,Drink,3,organic nonfat milk,1 cup,90 cal,130 mg,12 g,0 g,9 g,milk,no,U.S.A.,0.0
1,Hummus,Fridge,10,Cooked organic chickpeas,30 g,80,120 mg,6g,5g,2g,Sesame,no,U.S.A.,1.0
2,Graham crackers,Snack,12,Organic whole wheat flour,31 g,130,130 mg,24g,3g,2g,"wheat, milk, soy",no,U.S.A.,4.0
3,Cannellini beans,Canned/Dried,3,organic prepared cannellini beans,130 g,100,110 mg,19 g,0g,6g,NaN,no,U.S.A.,2.0
4,Raw shrimp,Freezer,2,raw shrimp,113 g,80,220 mg,0g,0.5g,19g,shrimp,no,U.S.A.,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Mustard,Fridge,6,Water,5 g,0 cal,55 mg,0 g,0 g,0 g,none,yes,U.S.A.,3.0
81,Vanilla Ice Cream,Freezer,11,Milk,89 g,200 cal,40 mg,21 g,12 g,3 g,"milk, soy",no,U.S.A.,2.0
82,Cheese Crackers,Snack,18,Flour,28 g,140 cal,260 mg,17 g,7 g,3 g,"milk, wheat, soy",no,U.S.A.,4.0
83,La Croix (fizzy water),Drink,1,Water,354.88 g,0 cal,0 mg,0 g,0 g,0 g,none,no,U.S.A.,2.0


In [399]:
def lower_data(row):
    global df
    def try_lower(x):
        try:
            return x.lower()
        except AttributeError:
            return x
    df[row] = df[row].apply(try_lower)
def if_then(row,cond,func):
    global df
    def try_func(x):
        try:
            if cond(x):
                return func(x)
            return x
        except:
            return x
    df[row] = df[row].apply(try_func)

def remove_stuff(row,s):
    global df
    def try_lower(x):
        try:
            return x.replace(s,"")
        except AttributeError:
            return x
    df[row] = df[row].apply(try_lower)
def cleanify(unit, row, strict=True,put_none_if_no_match=True):
    global df
    if strict:
        df = df[df[row].str.contains(unit)]
    def clean(s):
        #https://regexr.com/85git amazing
        if strict:
            r = re.compile(f"([0-9]+\ ?(\.|\/)?\ ?[0-9]*)\ ?{unit}")
        else:
            r = re.compile(f"([0-9]+\ ?(\.|\/)?\ ?[0-9]*)\ ?({unit})?")
        matches = re.match(r, s)
        if matches:
            return matches.group(1)
        if put_none_if_no_match:
            return None
        return s

    df[row] = df[row].apply(clean)

In [400]:
#cleanify("g","Serving Size", False)

In [401]:
cleanify("cal","Calories per serving", False)

In [402]:
cleanify("mg","Total Sodium", False)

In [403]:
cleanify("g","Total Carbohydrates", False)

In [404]:
cleanify("g","Total Fat", False)

In [405]:
cleanify("g","Total Protein", False)

In [406]:
lower_data("Item Name")
lower_data("Name of first (main) ingredient")
lower_data("What allergens does it contain?")
lower_data("Does it contain GMO crops?")
lower_data("Where was it made?")

In [407]:
remove_stuff("Where was it made?", ".") # for stuff like u.s.a


In [408]:
df = df.replace({"Does it contain GMO crops?": {
    "no":0,
    "yes":1,
}})
df = df.replace({"What kind of item is it?": {
    "Drink":1,
    "Fridge":2,
    "Snack":3,
    "Canned/Dried":4,
    "Freezer":2
}})


/var/folders/tw/cm45tsj50pn501zrkvnzcywm0000gp/T/ipykernel_74994/1302458478.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"What kind of item is it?": {


In [409]:
df = df.fillna("none")

In [410]:
if_then("What allergens does it contain?", lambda x: "n/a" in x, lambda x: "none")

In [411]:
df

,Item Name,What kind of item is it?,Total number of ingredients,Name of first (main) ingredient,Serving Size,Calories per serving,Total Sodium,Total Carbohydrates,Total Fat,Total Protein,What allergens does it contain?,Does it contain GMO crops?,Where was it made?,How Processed is it?\n(see instructions for ranking)
0,milk,1,3,organic nonfat milk,1 cup,90,130,12,0,9,milk,0,usa,0.0
1,hummus,2,10,cooked organic chickpeas,30 g,80,120,6,5,2,sesame,0,usa,1.0
2,graham crackers,3,12,organic whole wheat flour,31 g,130,130,24,3,2,"wheat, milk, soy",0,usa,4.0
3,cannellini beans,4,3,organic prepared cannellini beans,130 g,100,110,19,0,6,none,0,usa,2.0
4,raw shrimp,2,2,raw shrimp,113 g,80,220,0,0.5,19,shrimp,0,usa,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,mustard,2,6,water,5 g,0,55,0,0,0,none,1,usa,3.0
81,vanilla ice cream,2,11,milk,89 g,200,40,21,12,3,"milk, soy",0,usa,2.0
82,cheese crackers,3,18,flour,28 g,140,260,17,7,3,"milk, wheat, soy",0,usa,4.0
83,la croix (fizzy water),1,1,water,354.88 g,0,0,0,0,0,none,0,usa,2.0


In [412]:
df.to_csv(os.path.expanduser("~/Documents/GitHub/q1-cs/foodstuffs/emerald_cleaned.csv"))